In [61]:
import requests
import pandas as pd
import numpy as np

In [62]:
def get_counties():
    url_county = 'https://prezenta.roaep.ro/locale27092020/data/json/sicpv/lists/counties.json'
    r = requests.get(url=url_county)
    return r.json()

In [63]:
def get_uat_of_county(county_key):
    url_uat = 'https://prezenta.roaep.ro/locale27092020/data/json/sicpv/lists/uats_' + county_key + '.json'
    r = requests.get(url=url_uat)
    return r.json()

In [64]:
def get_commune_of_county(county_key):
    url_uat = 'https://prezenta.roaep.ro/locale27092020/data/json/sicpv/lists/localities_' + county_key + '.json'
    r = requests.get(url=url_uat)
    return r.json()

In [65]:
def get_precinct_of_county(county_key):
    url_uat = 'https://prezenta.roaep.ro/locale27092020/data/json/sicpv/lists/precincts_' + county_key + '.json'
    r = requests.get(url=url_uat)
    return r.json()

In [101]:
def get_results_of_county(county_key):
    url_uat = 'https://prezenta.roaep.ro/locale27092020/data/json/sicpv/pv/pv_' + county_key + '_part.json'
    r = requests.get(url=url_uat)
    return r.json()

In [112]:
def get_row_for_category(pres_of_uat, results, data_key, new_row):
    uat_calculated_results = {}
    uat_sum_all_parties = 0
    for precinct in pres_of_uat:
        pre_id = str(precinct['id']);
        
        if pre_id not in results:
            continue
        
        lt = results[pre_id]['votes']
        for party in lt:
            party_name = party[data_key]
            party_votes = party['votes']
            uat_sum_all_parties += int(party_votes)
            if party_name in uat_calculated_results:
                uat_calculated_results[party_name] += int(party_votes)
            else:
                uat_calculated_results[party_name] = int(party_votes)

    # add columns if not exists
    for key in uat_calculated_results.keys():
        if key not in df:
            df[key] = np.nan
        new_row[key] = uat_calculated_results[key]    
        
    return new_row, uat_sum_all_parties


In [113]:
def add_row_by_result_type(result_type, data_key, county_name, uat_name, pres_of_uat, results, df):
    new_row = {'County':county_name, 'UAT':uat_name, 'TIP': result_type, 'Total': 0}     
    new_row, uat_total_sum = get_row_for_category(pres_of_uat, results, data_key, new_row)
    new_row['Total'] = uat_total_sum
    df = df.append(new_row, ignore_index=True)
    return df

In [114]:
def get_dataframe_of_county(county_name, uats, locs, pres, results_p, results_cl, results_cj, results_pcj, df):
    
    for item in uats:
        
        uat_id = item['id']
        uat_name = item['name']
        siruta = item['siruta']
        #print(uat_name)

        locs_of_uat = [loc for loc in locs if loc['uat_id'] == str(uat_id)]
        #print(locs_of_uat)

        pres_of_uat = [per for per in pres if per['uat_id'] == uat_id]
        #print(pres_of_uat)

        df = add_row_by_result_type('P', 'party', county_name, uat_name, pres_of_uat, results_p, df)
        df = add_row_by_result_type('CL', 'candidate', county_name, uat_name, pres_of_uat, results_cl, df)
        df = add_row_by_result_type('CJ', 'candidate', county_name, uat_name, pres_of_uat, results_cj, df)
        df = add_row_by_result_type('PCJ', 'party', county_name, uat_name, pres_of_uat, results_pcj, df)
        
        # result_of_uat = [res for res in results if res['']]
        #new_row = {'County':county_name, 'UAT':uat_name, 'TIP': 'P'}     
        #new_row = get_row_for_category(pres_of_uat, results_p, 'p_', 'party', new_row)
        #df = df.append(new_row, ignore_index=True)
        
        #new_row = get_row_for_category(pres_of_uat, results_cl, 'cl_', 'candidate', new_row)
        #new_row = get_row_for_category(pres_of_uat, results_cj, 'cj_', 'candidate', new_row)
        #new_row = get_row_for_category(pres_of_uat, results_pcj, 'pcj_', 'candidate', new_row)


        # print(new_row)
        # append row to the dataframe
        #df = df.append(new_row, ignore_index=True)
    return df


In [118]:
df = pd.DataFrame(columns=['County','UAT'])

In [116]:
counties = get_counties()

for item in counties:
    county_name = item['name']
    county_key = item['code'].lower()
    print(county_name)
    
    uats = get_uat_of_county(county_key)
    locs = get_commune_of_county(county_key)
    pres = get_precinct_of_county(county_key)
    results = get_results_of_county(county_key)
    results_p = results['stages']['PART']['scopes']['PRCNCT']['categories']['P']['table']
    results_cl = results['stages']['PART']['scopes']['PRCNCT']['categories']['CL']['table']
    results_cj = results['stages']['PART']['scopes']['PRCNCT']['categories']['CJ']['table']
    results_pcj = results['stages']['PART']['scopes']['PRCNCT']['categories']['PCJ']['table']

  
    df = get_dataframe_of_county(county_name, uats, locs, pres, results_p, results_cl, results_cj, results_pcj, df)

ALBA
ARAD
ARGEŞ
BACĂU
BIHOR
BISTRIŢA-NĂSĂUD
BOTOŞANI
BRĂILA
BRAŞOV
BUZĂU
CĂLĂRAŞI
CARAŞ-SEVERIN
CLUJ
CONSTANŢA
COVASNA
DÂMBOVIŢA
DOLJ
GALAŢI
GIURGIU
GORJ
HARGHITA
HUNEDOARA
IALOMIŢA
IAŞI
ILFOV
MARAMUREŞ
MEHEDINŢI
MUNICIPIUL BUCUREŞTI
MUREŞ
NEAMŢ
OLT
PRAHOVA
SĂLAJ
SATU MARE
SIBIU
SUCEAVA
TELEORMAN
TIMIŞ
TULCEA
VÂLCEA
VASLUI
VRANCEA


In [117]:
df.to_csv('locale2020_provizorii.csv', header=True, index=False, encoding='utf-8-sig')